## VaR

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
# Load SPY data function
def fetch_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Close']

In [3]:
# Set parameters
ticker = "SPY"
start_date = "2022-01-01"
end_date = "2023-12-31"
alpha = 0.05  # 95% confidence level

# Load data
data = fetch_data(ticker, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [4]:
data_spy = data.to_frame()

In [5]:
# Function to calculate VaR
def calculate_var(data, window_size, confidence_level, num_simulations):
    """
    Calculate the daily VaR using Monte Carlo method.

    Parameters.
    data (pd.DataFrame): DataFrame containing the closing price, index should be date.
    window_size (int): sliding window size in days.
    confidence_level (float): Confidence level of the VaR.
    num_simulations (int): number of Monte Carlo simulations.

    Returns.
    pd.DataFrame: Raw data plus daily VaR.
    """
    # Calculate return
    data['Returns'] = data['Close'].pct_change()

    # Initialize VaR
    vaR_series = pd.Series(np.nan, index=data.index)

    # Calcualte VaR based on window size
    for i in range(window_size, len(data)):
        historical_returns = data['Returns'].iloc[i-window_size:i].dropna()
        if len(historical_returns) < window_size:
            continue  # If historical data is not enough for window size, continue

        mean_returns = historical_returns.mean()
        std_dev = historical_returns.std()

        simulated_returns = np.random.normal(mean_returns, std_dev, num_simulations)
        simulated_losses = -simulated_returns
        VaR = np.percentile(simulated_losses, 100 * (1 - confidence_level))
        vaR_series[i] = VaR 

    return vaR_series

# Function to calculate strike price
def calculate_strike_prices(data, multipliers):
    strikes_data = {}
    for multiplier in multipliers:
        strike_prices = data['Close'] * (1 + data['VaR']) * multiplier
        strikes_data[f'Strike_x{multiplier}'] = strike_prices
    return pd.DataFrame(strikes_data, index=data.index)

In [6]:
# Set parameters (Set 2 weeks as maturity)
window_size = 10
confidence_level = 0.95
num_simulations = 10000

# Parameters to calculate different strike price
multipliers = [1.1, 1.05, 1, 0.95, 0.9]

# Calculate VaR
data_spy['VaR'] = calculate_var(data_spy, window_size, confidence_level, num_simulations)

In [7]:
data_spy_var = data_spy.dropna()

In [8]:
data_strikeprice = calculate_strike_prices(data_spy_var, multipliers)

In [9]:
data_strikeprice

,Strike_x1.1,Strike_x1.05,Strike_x1,Strike_x0.95,Strike_x0.9
Date,,,,,
2022-01-19,491.723331,469.372271,447.021210,424.670150,402.319089
2022-01-20,486.537113,464.421789,442.306466,420.191143,398.075820
2022-01-21,477.444743,455.742709,434.040675,412.338642,390.636608
2022-01-24,479.639017,457.837243,436.035470,414.233696,392.431923
2022-01-25,472.624639,451.141700,429.658762,408.175824,386.692886
...,...,...,...,...,...
2023-12-22,512.701257,489.396654,466.092051,442.787449,419.482846
2023-12-26,514.988706,491.580128,468.171551,444.762973,421.354396
2023-12-27,516.073488,492.615602,469.157716,445.699831,422.241945


In [12]:
data_strikeprice.to_csv('./VaR_strikeprice.csv', index=True)